In [1]:
import ast

import pandas as pd
import janitor
import altair as alt
from pydash import py_

from funcs.paths import paths
print(paths)

{'root': PosixPath('/data/ik18445/projects/biorxiv-medrxiv-tdm'), 'data_root': PosixPath('/data/ik18445/projects/biorxiv-medrxiv-tdm/data'), 'raw_data_dir': PosixPath('/data/ik18445/projects/biorxiv-medrxiv-tdm/data/local-source-data'), 'examples_data_dir': PosixPath('/data/ik18445/projects/biorxiv-medrxiv-tdm/data/local-source-data/examples'), 'output': PosixPath('/data/ik18445/projects/biorxiv-medrxiv-tdm/output'), 'results': PosixPath('/data/ik18445/projects/biorxiv-medrxiv-tdm/output/results'), 'artifacts': PosixPath('/data/ik18445/projects/biorxiv-medrxiv-tdm/output/artifacts'), 'tmp_output': PosixPath('/data/ik18445/projects/biorxiv-medrxiv-tdm/output/tmp')}


In [2]:
INPUT_DIR = paths["results"] / "metadata_extract"
assert INPUT_DIR.exists(), INPUT_DIR
OUTPUT_DIR = paths["results"]
assert OUTPUT_DIR.exists(), OUTPUT_DIR

----

# Collect mass extract results

In [3]:
input_file = INPUT_DIR / "meta_info.csv"
assert input_file.exists(), input_file

meta_info = pd.read_csv(input_file)
meta_info

,input_dir,output_results_file,output_error_file
0,/data/ik18445/projects/biorxiv-medrxiv-tdm/dat...,/data/ik18445/projects/biorxiv-medrxiv-tdm/out...,/data/ik18445/projects/biorxiv-medrxiv-tdm/out...
1,/data/ik18445/projects/biorxiv-medrxiv-tdm/dat...,/data/ik18445/projects/biorxiv-medrxiv-tdm/out...,/data/ik18445/projects/biorxiv-medrxiv-tdm/out...
2,/data/ik18445/projects/biorxiv-medrxiv-tdm/dat...,/data/ik18445/projects/biorxiv-medrxiv-tdm/out...,/data/ik18445/projects/biorxiv-medrxiv-tdm/out...
3,/data/ik18445/projects/biorxiv-medrxiv-tdm/dat...,/data/ik18445/projects/biorxiv-medrxiv-tdm/out...,/data/ik18445/projects/biorxiv-medrxiv-tdm/out...
4,/data/ik18445/projects/biorxiv-medrxiv-tdm/dat...,/data/ik18445/projects/biorxiv-medrxiv-tdm/out...,/data/ik18445/projects/biorxiv-medrxiv-tdm/out...
...,...,...,...
150,/data/ik18445/projects/biorxiv-medrxiv-tdm/dat...,/data/ik18445/projects/biorxiv-medrxiv-tdm/out...,/data/ik18445/projects/biorxiv-medrxiv-tdm/out...
151,/data/ik18445/projects/biorxiv-medrxiv-tdm/dat...,/data/ik18445/projects/biorxiv-medrxiv-tdm/out...,/data/ik18445/projects/biorxiv-medrxiv-tdm/out...
152,/data/ik18445/projects/biorxiv-medrxiv-tdm/dat...,/data/ik18445/projects/biorxiv-medrxiv-tdm/out...,/data/ik18445/projects/biorxiv-medrxiv-tdm/out...
153,/data/ik18445/projects/biorxiv-medrxiv-tdm/dat...,/data/ik18445/projects/biorxiv-medrxiv-tdm/out...,/data/ik18445/projects/biorxiv-medrxiv-tdm/out...


In [4]:
metadata_results_nested = []
for _ in meta_info["output_results_file"]:
    try:
        df = pd.read_csv(_)
        metadata_results_nested.append(df)
    except:
        continue
metadata_results = pd.concat(metadata_results_nested).reset_index(drop=True)
    
metadata_results.info()
metadata_results

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 322228 entries, 0 to 322227
Data columns (total 9 columns):
 #   Column               Non-Null Count   Dtype  
---  ------               --------------   -----  
 0   zip_path             322228 non-null  object 
 1   full_text_file       322228 non-null  object 
 2   doi                  322228 non-null  object 
 3   title                322227 non-null  object 
 4   version              322228 non-null  float64
 5   category             322221 non-null  object 
 6   year_month           322227 non-null  object 
 7   preprint_source      322228 non-null  object 
 8   publish_destination  22 non-null      object 
dtypes: float64(1), object(8)
memory usage: 22.1+ MB


,zip_path,full_text_file,doi,title,version,category,year_month,preprint_source,publish_destination
0,biorxiv/Back_Content/Batch_10/2b03c3ec-6c00-10...,content/083659.xml,10.1101/083659,Structural and Functional analysis of the GABA...,1.1,Biochemistry,"{'year': 2016, 'month': 10}",BIORXIV,NaN
1,biorxiv/Back_Content/Batch_10/be21e297-6bf6-10...,content/088120.xml,10.1101/088120,Electroceutical disinfection strategies impair...,1.1,Bioengineering,"{'year': 2016, 'month': 11}",BIORXIV,NaN
2,biorxiv/Back_Content/Batch_10/4362745b-6bf8-10...,content/090647.xml,10.1101/090647,"The emergence, evolution, and diversification ...",1.1,Evolutionary Biology,"{'year': 2016, 'month': 12}",BIORXIV,NaN
3,biorxiv/Back_Content/Batch_10/407b7537-6bfc-10...,content/054858.xml,10.1101/054858,NCS1 regulates Ca2+-Dependent Focal Exocytosis...,1.2,Cell Biology,"{'year': 2016, 'month': 11}",BIORXIV,NaN
4,biorxiv/Back_Content/Batch_10/e8854b54-6c05-10...,content/086934.xml,10.1101/086934,Spike interval coding of translatory optic flo...,1.1,Neuroscience,"{'year': 2016, 'month': 11}",BIORXIV,NaN
...,...,...,...,...,...,...,...,...,...
322223,medrxiv/Current_Content/March_2022/8eef593f-6e...,content/22272775.xml,10.1101/2022.03.22.22272775,Risk of death following SARS-CoV-2 infection o...,1.1,Epidemiology,"{'year': 2022, 'month': 3}",MEDRXIV,NaN
322224,medrxiv/Current_Content/March_2022/338749a2-6d...,content/22272553.xml,10.1101/2022.03.18.22272553,15-month follow-up of anti-spike receptor bind...,1.1,Infectious Diseases (except HIV/AIDS),"{'year': 2022, 'month': 3}",MEDRXIV,NaN
322225,medrxiv/Current_Content/March_2022/446c7d6f-7a...,content/22269512.xml,10.1101/2022.01.27.22269512,Investigation of air change rate and aerosol b...,1.2,Occupational and Environmental Health,"{'year': 2022, 'month': 3}",MEDRXIV,NaN
322226,medrxiv/Current_Content/March_2022/16ae3262-6e...,content/21257682.xml,10.1101/2021.05.23.21257682,Type 2 diabetes mellitus accelerates brain agi...,1.3,Endocrinology,"{'year': 2022, 'month': 3}",MEDRXIV,NaN


In [5]:
output_file = OUTPUT_DIR / "metadata_results.csv"
metadata_results.to_csv(output_file, index=False)

---

# Load

In [6]:
input_file = OUTPUT_DIR / "metadata_results.csv"
metadata_results = pd.read_csv(input_file)
metadata_results = metadata_results \
    .assign(
        year_month=lambda df: df["year_month"].apply(
            lambda e: ast.literal_eval(e) if not pd.isnull(e) else e
        )
    ) \
    .dropna(subset=["year_month"]) \
    .assign(
        time=lambda df: df["year_month"].apply(
            lambda e: "{year}-{month}".format(year=e["year"], month=e["month"]) 
        ).pipe(pd.to_datetime)
    )
metadata_results = metadata_results[
    metadata_results["preprint_source"].isin(["BIORXIV", "MEDRXIV"])
]

/tmp/ipykernel_60250/2768983954.py:2: DtypeWarning: Columns (8) have mixed types. Specify dtype option on import or set low_memory=False.
  metadata_results = pd.read_csv(input_file)


# Diagnostics, sanity checks

In [7]:
metadata_results[~metadata_results["publish_destination"].isna()]["publish_destination"]

1287         YNIMG
1434          PONE
1464         peerj
1578          BCAN
1675           mbc
1744         elife
1773        Genome
1791         elife
1983          GBIO
2287         elife
2545         elife
10322         pnas
10336        YNIMG
10426        elife
10799        YNIMG
10847        elife
18341          msb
22984    PCOMPBIOL
23865    PCOMPBIOL
46715         giga
47347    PMEDICINE
52249        YNIMG
Name: publish_destination, dtype: object

In [8]:
metadata_results[["doi", "version"]].dropna() \
    .groupby(["doi"]) \
    .size().sort_values(ascending=False)

doi
10.1101/2020.07.09.20143164    26
10.1101/290825                 25
10.1101/066423                 19
10.1101/016840                 19
10.1101/2020.05.26.104687      16
                               ..
10.1101/871475                  1
10.1101/871574                  1
10.1101/871640                  1
10.1101/871665                  1
10.1101/871707                  1
Length: 237631, dtype: int64

# Analysis

## total count

In [9]:
df_ts_total = metadata_results \
    [["time", "preprint_source"]].dropna() \
    .groupby(["time", "preprint_source"]).size() \
    .rename("count").to_frame().reset_index(drop=False)
df_ts_total

,time,preprint_source,count
0,2013-10-01,BIORXIV,5
1,2013-11-01,BIORXIV,64
2,2013-12-01,BIORXIV,69
3,2014-01-01,BIORXIV,71
4,2014-02-01,BIORXIV,85
...,...,...,...
159,2023-03-01,MEDRXIV,1270
160,2023-04-01,BIORXIV,4083
161,2023-04-01,MEDRXIV,968
162,2023-05-01,BIORXIV,4232


In [56]:
df_combined = pd.concat([
    df_ts_total \
        .groupby("time")["count"] \
        .sum().to_frame().reset_index(drop=False) \
        .assign(preprint_source="combined"),
    df_ts_total
]).reset_index(drop=True)


p = alt.Chart(
    df_combined
).mark_line() \
    .encode(
        alt.X("time:T", axis=alt.Axis(format="%Y-%m")),
        y="count:Q",
        color="preprint_source:N"
    )
rule = alt.Chart(pd.DataFrame({
    "time": ["2020-02-01", "2020-05-01"],
    "color": ["black", "red"]
})).mark_rule(
    strokeDash=[8, 4]
).encode(
    x="time:T",
    color=alt.Color("color:N", scale=None),
    size=alt.value(2)
)

(p + rule).properties(
        width=800,
        height=400
    )

alt.LayerChart(...)

## categories

In [32]:
df_cat = metadata_results \
    [["category", "preprint_source"]].dropna() \
    .groupby(["category", "preprint_source"]).size() \
    .rename("count").to_frame().reset_index(drop=False) \
    .sort_values(["count"], ascending=False)
df_cat

,category,preprint_source,count
56,Neuroscience,BIORXIV,50610
8,Bioinformatics,BIORXIV,24723
51,Microbiology,BIORXIV,23545
29,Evolutionary Biology,BIORXIV,15901
36,Genomics,BIORXIV,14658
...,...,...,...
21,ECOLOGY,BIORXIV,1
11,CELL BIOLOGY,BIORXIV,1
10,CANCER BIOLOGY,BIORXIV,1
77,Psychiatry and Clinical Psychology,BIORXIV,1


In [33]:
alt.Chart(df_cat).mark_bar() \
    .encode(
        y="category:N",
        x="count:Q",
        tooltip=["category", "count"]
    ) \
    .properties(
        width=400,
        height=800
    ) \
    .facet(
        row="preprint_source:N"
    ) \
    .interactive()

alt.FacetChart(...)

## TS distribution of categories

In [13]:
df_ts_cat = metadata_results \
    [["time", "category", "preprint_source"]].dropna()

df_ts_cat

,time,category,preprint_source
0,2016-10-01,Biochemistry,BIORXIV
1,2016-11-01,Bioengineering,BIORXIV
2,2016-12-01,Evolutionary Biology,BIORXIV
3,2016-11-01,Cell Biology,BIORXIV
4,2016-11-01,Neuroscience,BIORXIV
...,...,...,...
322223,2022-03-01,Epidemiology,MEDRXIV
322224,2022-03-01,Infectious Diseases (except HIV/AIDS),MEDRXIV
322225,2022-03-01,Occupational and Environmental Health,MEDRXIV
322226,2022-03-01,Endocrinology,MEDRXIV


In [14]:
df_ts_cat_sum = df_ts_cat \
    .groupby(["time", "category", "preprint_source"]).size() \
    .rename("count").to_frame().reset_index(drop=False)
df_ts_cat_sum

,time,category,preprint_source,count
0,2013-10-01,Cancer Biology,BIORXIV,1
1,2013-10-01,Evolutionary Biology,BIORXIV,1
2,2013-10-01,Genetics,BIORXIV,1
3,2013-10-01,Genomics,BIORXIV,1
4,2013-10-01,Synthetic Biology,BIORXIV,1
...,...,...,...,...
5076,2023-05-01,Systems Biology,BIORXIV,106
5077,2023-05-01,Toxicology,MEDRXIV,1
5078,2023-05-01,Transplantation,MEDRXIV,5
5079,2023-05-01,Urology,MEDRXIV,5


In [24]:
biorxiv_ts_cat_sum = df_ts_cat_sum[df_ts_cat_sum["preprint_source"] == "BIORXIV"]
top_sum = biorxiv_ts_cat_sum.groupby(["category"])["count"].sum().sort_values(ascending=False)
top_15_cats = top_sum[:15].index.to_list()
biorxiv_ts_cat_sum = biorxiv_ts_cat_sum \
    .assign(category=lambda df: df["category"].apply(lambda e: e if e in top_15_cats else "Others"))
biorxiv_ts_cat_sum

,time,category,preprint_source,count
0,2013-10-01,Cancer Biology,BIORXIV,1
1,2013-10-01,Evolutionary Biology,BIORXIV,1
2,2013-10-01,Genetics,BIORXIV,1
3,2013-10-01,Genomics,BIORXIV,1
4,2013-10-01,Others,BIORXIV,1
...,...,...,...,...
5063,2023-05-01,Plant Biology,BIORXIV,144
5071,2023-05-01,Others,BIORXIV,16
5075,2023-05-01,Others,BIORXIV,46
5076,2023-05-01,Others,BIORXIV,106


In [30]:
p_biorxiv_ts_cat_sum = alt.Chart(biorxiv_ts_cat_sum).mark_bar() \
    .encode(
        alt.X("time:T", axis=alt.Axis(format="%Y-%m")),
        y="count:Q",
        color="category:N",
    ) \
    .properties(
        width=800,
        height=400
    )
p_biorxiv_ts_cat_sum

alt.Chart(...)

In [29]:
medrxiv_ts_cat_sum = df_ts_cat_sum[df_ts_cat_sum["preprint_source"] == "MEDRXIV"]
top_sum = medrxiv_ts_cat_sum.groupby(["category"])["count"].sum().sort_values(ascending=False)
top_15_cats = top_sum[:15].index.to_list()
medrxiv_ts_cat_sum = medrxiv_ts_cat_sum \
    .assign(category=lambda df: df["category"].apply(lambda e: e if e in top_15_cats else "Others"))
medrxiv_ts_cat_sum

,time,category,preprint_source,count
1631,2019-06-01,Cardiovascular Medicine,MEDRXIV,2
1634,2019-06-01,Others,MEDRXIV,1
1638,2019-06-01,Epidemiology,MEDRXIV,3
1640,2019-06-01,Genetic and Genomic Medicine,MEDRXIV,1
1643,2019-06-01,Health Informatics,MEDRXIV,3
...,...,...,...,...
5073,2023-05-01,Others,MEDRXIV,8
5074,2023-05-01,Others,MEDRXIV,10
5077,2023-05-01,Others,MEDRXIV,1
5078,2023-05-01,Others,MEDRXIV,5


In [31]:
p_medrxiv_ts_cat_sum = alt.Chart(medrxiv_ts_cat_sum).mark_bar() \
    .encode(
        alt.X("time:T", axis=alt.Axis(format="%Y-%m")),
        y="count:Q",
        color="category:N",
    ) \
    .properties(
        width=800,
        height=400
    )
p_medrxiv_ts_cat_sum

alt.Chart(...)

## preprint versions

In [59]:
df_versions = metadata_results[["doi", "version"]].dropna() \
    .groupby(["doi"]).size().rename("num_versions") \
    .sort_values(ascending=False).to_frame().reset_index(drop=False)
df_versions

,doi,num_versions
0,10.1101/2020.07.09.20143164,26
1,10.1101/290825,25
2,10.1101/066423,19
3,10.1101/016840,19
4,10.1101/2020.05.26.104687,16
...,...,...
237626,10.1101/871475,1
237627,10.1101/871574,1
237628,10.1101/871640,1
237629,10.1101/871665,1


In [60]:
df_versions_sum = df_versions.groupby("num_versions") \
    .size().rename("count").to_frame().reset_index(drop=False)
df_versions_sum

,num_versions,count
0,1,177751
1,2,43092
2,3,11806
3,4,3328
4,5,999
5,6,374
6,7,146
7,8,58
8,9,35
9,10,13
